In [528]:
import pandas as pd

# Mapowania

In [529]:
mapowanie_pkd_2007_2025 = pd.read_excel(r"..\data\mapowanie_pkd.xlsx", sheet_name="MAP_PKD_2007_2025")
# mapowanie_pkd_2025_2007 = pd.read_excel(r"..\data\mapowanie_pkd.xlsx", sheet_name="MAP_PKD_2025_2007")
pkd_2025 = pd.read_excel(r"..\data\mapowanie_pkd.xlsx", sheet_name="PKD_2025")
# pkd_2007 = pd.read_excel(r"..\data\mapowanie_pkd.xlsx", sheet_name="PKD_2007")

In [530]:
mapowanie_pkd_2007_2025_with_og = pd.concat([
    mapowanie_pkd_2007_2025,
    pd.DataFrame({'symbol_2007': ['OG'], 'symbol_2025': ['OG']})
], ignore_index=True)

In [531]:
pkd_2025

,typ,symbol,nazwa
0,SEKCJA,A,"ROLNICTWO, LEŚNICTWO I RYBACTWO"
1,DZIAŁ,01,"UPRAWY ROLNE, CHÓW I HODOWLA ZWIERZĄT, ŁOWIECT..."
2,GRUPA,01.1,Uprawy rolne inne niż wieloletnie
3,KLASA,01.11,"Uprawa zbóż innych niż ryż, roślin strączkowyc..."
4,PODKLASA,01.11.Z,"Uprawa zbóż innych niż ryż, roślin strączkowyc..."
...,...,...,...
1770,SEKCJA,V,ORGANIZACJE I ZESPOŁY EKSTERYTORIALNE
1771,DZIAŁ,99,ORGANIZACJE I ZESPOŁY EKSTERYTORIALNE
1772,GRUPA,99.0,Organizacje i zespoły eksterytorialne
1773,KLASA,99.00,Organizacje i zespoły eksterytorialne


In [532]:
pkd_2025_with_og = pd.concat([
    pkd_2025,
    pd.DataFrame({'typ': ['OGÓŁEM'], 'symbol': ['OG'], 'nazwa': ['OGÓŁEM']})
], ignore_index=True)
pkd_2025_with_og

,typ,symbol,nazwa
0,SEKCJA,A,"ROLNICTWO, LEŚNICTWO I RYBACTWO"
1,DZIAŁ,01,"UPRAWY ROLNE, CHÓW I HODOWLA ZWIERZĄT, ŁOWIECT..."
2,GRUPA,01.1,Uprawy rolne inne niż wieloletnie
3,KLASA,01.11,"Uprawa zbóż innych niż ryż, roślin strączkowyc..."
4,PODKLASA,01.11.Z,"Uprawa zbóż innych niż ryż, roślin strączkowyc..."
...,...,...,...
1771,DZIAŁ,99,ORGANIZACJE I ZESPOŁY EKSTERYTORIALNE
1772,GRUPA,99.0,Organizacje i zespoły eksterytorialne
1773,KLASA,99.00,Organizacje i zespoły eksterytorialne
1774,PODKLASA,99.00.Z,Organizacje i zespoły eksterytorialne


# Upadlosci - KRZ_PKD
krz_pkd.csv - przypisanie liczby upadłości firm z Krajowego Rejestru Zadłużonych do PKD
(2007) od 2018 roku -
**poziom podklasy**

In [533]:
upadlosci = pd.read_csv(r"..\data\krz_pkd.csv", sep=";", encoding="utf-8")

In [534]:
upadlosci["pkd_formatted"] = (
    upadlosci["pkd"]
    .str.strip()
    .str.upper()
    .str.replace(r"^(\d{2})(\d{2})([A-Z])$", r"\1.\2.\3", regex=True)
)

In [535]:
upadlosci["pkd_formatted_no_letter"] = upadlosci["pkd_formatted"].str.replace(r"\.[A-Z]$", "", regex=True)
upadlosci["pkd_letter"] = upadlosci["pkd_formatted"].str.extract(r"\.([A-Z])$")

upadlosci_mapped = upadlosci.merge(
    mapowanie_pkd_2007_2025[["symbol_2007", "symbol_2025"]],
    how="left",
    left_on="pkd_formatted_no_letter",
    right_on="symbol_2007"
)
unmapped_mask = upadlosci_mapped["symbol_2025"].isna()
if unmapped_mask.any():
    missing_codes = (
        upadlosci_mapped.loc[unmapped_mask, "pkd_formatted_no_letter"]
        .dropna()
        .unique()
        .tolist()
    )
    print(f"Warning: Niezmapowane kody PKD 2007: {missing_codes}")
    upadlosci_mapped = upadlosci_mapped[~unmapped_mask]


In [536]:
upadlosci_mapped["pkd_2025"] = upadlosci_mapped["symbol_2025"] + '.' + upadlosci_mapped["pkd_letter"]

In [537]:
upadlosci_final = upadlosci_mapped[["rok", "pkd_2025", "liczba_upadlosci"]]

In [538]:
upadlosci_final

,rok,pkd_2025,liczba_upadlosci
0,2018,01.11.Z,6
1,2018,01.13.Z,4
2,2018,01.19.Z,2
3,2018,01.24.Z,1
4,2018,01.29.Z,1
...,...,...,...
2540,2024,96.21.Z,12
2541,2024,96.22.Z,12
2542,2024,96.30.Z,1
2543,2024,93.13.Z,1


# Wskaznik finansowy
wszystkie poziomy oprócz podklas

In [539]:
wskaznik_finansowy = pd.read_csv(r"..\data\wsk_fin.csv", sep=";", encoding="utf-8")

In [540]:
#  OG - ogółem; SEK_A - sekctor; 
# wskaznik_finansowy

In [541]:
wskaznik_finansowy["PKD_formatted"] = wskaznik_finansowy["PKD"].str.replace(r"^SEK_", "", regex=True).str.rstrip('.')

In [542]:
wskaznik_finansowy = wskaznik_finansowy.drop(columns=["NAZWA_PKD", "NUMER_NAZWA_PKD","PKD"])

In [543]:
wskaznik_finansowy_mapped = wskaznik_finansowy.join(
    mapowanie_pkd_2007_2025_with_og[["symbol_2007", "symbol_2025"]].set_index("symbol_2007"),
    on="PKD_formatted"
)
unmapped_mask = wskaznik_finansowy_mapped["symbol_2025"].isna()
if unmapped_mask.any():
    missing_codes = (
        wskaznik_finansowy_mapped.loc[unmapped_mask, "PKD_formatted"]
        .dropna()
        .unique()
        .tolist()
    )
    raise ValueError(f"Niezmapowane kody PKD 2007: {missing_codes}")

In [544]:
wskaznik_finansowy_mapped = wskaznik_finansowy_mapped.drop(columns=["PKD_formatted"])

In [545]:
wskaznik_finansowy_transposed = wskaznik_finansowy_mapped.melt(
    id_vars=['symbol_2025', 'WSKAZNIK'],
    var_name='rok',
    value_name='wartosc'
)
wskaznik_finansowy_transposed['rok'] = wskaznik_finansowy_transposed['rok'].astype(int)

In [546]:
wskaznik_finansowy_transposed['wartosc'] = wskaznik_finansowy_transposed['wartosc'].replace('bd', pd.NA)

In [547]:
wskaznik_finansowy_final = wskaznik_finansowy_transposed.rename(columns={'symbol_2025': 'pkd_2025'})

In [548]:
wskaznik_finansowy_final


,pkd_2025,WSKAZNIK,rok,wartosc
0,OG,EN Liczba jednostek gospodarczych,2005,"46 396,00"
1,OG,PEN Liczba rentownych jednostek gospodarczych,2005,"35 860,00"
2,OG,GS Przychody ogółem,2005,"1 679 774,30"
3,OG,PNPM Przychody netto,2005,<NA>
4,OG,GS (I) Przychody netto ze sprzedaży i zrównane...,2005,"1 614 314,24"
...,...,...,...,...
418555,99.0,STL Zobowiązania krótkoterminowe,2024,<NA>
418556,99.0,LTC Długoterminowe kredyty bankowe,2024,<NA>
418557,99.0,STC Krótkoterminowe kredyty bankowe,2024,<NA>
418558,99.0,INV Zapasy,2024,<NA>


In [549]:
upadlosci_final['WSKAZNIK'] = 'Upadłość'
upadlosci_final = upadlosci_final.rename(columns={'liczba_upadlosci': 'wartosc'})

C:\Users\tusiu\AppData\Local\Temp\ipykernel_2196\2794861416.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  upadlosci_final['WSKAZNIK'] = 'Upadłość'


In [550]:
upadlosci_final

,rok,pkd_2025,wartosc,WSKAZNIK
0,2018,01.11.Z,6,Upadłość
1,2018,01.13.Z,4,Upadłość
2,2018,01.19.Z,2,Upadłość
3,2018,01.24.Z,1,Upadłość
4,2018,01.29.Z,1,Upadłość
...,...,...,...,...
2540,2024,96.21.Z,12,Upadłość
2541,2024,96.22.Z,12,Upadłość
2542,2024,96.30.Z,1,Upadłość
2543,2024,93.13.Z,1,Upadłość


In [551]:
import os
import pandas as pd

def read_quarterly_info(folder_path, sheet_name='Tabl 4'):
    """
    Reads all Excel files from the specified folder and extracts the specified sheet.
    Finds the header row starting with "Sekcja" in the first column.
    Raises an exception if the sheet is not found in any of the files.
    """
    all_data = []
    # List all xlsx files, ignoring temporary files
    files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx') and not f.startswith('~$')]
    
    if not files:
        print(f"No Excel files found in {folder_path}")
        return pd.DataFrame()

    for file in files:
        file_path = os.path.join(folder_path, file)
        
        # Load the Excel file to check sheet names
        xl = pd.ExcelFile(file_path)
        
        target_sheet = None
        # Define possible sheet names
        candidates = [sheet_name]
        if sheet_name == 'Tabl 4':
            candidates.append('Tabl. 4')
        
        for candidate in candidates:
            if candidate in xl.sheet_names:
                target_sheet = candidate
                break
        
        if target_sheet is None:
            raise ValueError(f"Sheet '{sheet_name}' (or variants {candidates}) not found in file '{file}'. Available sheets: {xl.sheet_names}")
            
        # Read the data without header first to find the "Sekcja" row
        df_temp = pd.read_excel(file_path, sheet_name=target_sheet, header=None)
        
        header_idx = None
        for idx, row in df_temp.iterrows():
            first_col_val = row.iloc[0]
            if isinstance(first_col_val, str) and "Sekcja" in first_col_val:
                header_idx = idx
                break
        
        if header_idx is not None:
            df = pd.read_excel(file_path, sheet_name=target_sheet, header=header_idx)
        else:
            print(f"Warning: 'Sekcja' not found in first column of {file}, using default header.")
            df = pd.read_excel(file_path, sheet_name=target_sheet)
        
        # Add a column to identify the source file (optional but useful)
        df['Source_File'] = file
        
        all_data.append(df)

    if all_data:
        return pd.concat(all_data, ignore_index=True)
    else:
        return pd.DataFrame()

# Define the path to the folder
folder_path = os.path.join('..', 'data', 'external', '_Full Kwartalna informacja o podmiotach gospodarki narodowej w rejestrze REGON')
# Run the function and show results
try:
    df_quarterly = read_quarterly_info(folder_path, sheet_name='Tabl 4')
    
    # Filter out rows where Sekcja is 'Brak PKD'
    df_quarterly = df_quarterly[df_quarterly['Sekcja'] != 'Brak PKD']
    
    # Clean Sekcja - all values which contains POLSKA should be replaced with OG
    df_quarterly.loc[df_quarterly['Sekcja'].str.contains('POLSKA', na=False), 'Sekcja'] = 'OG'

    # Consolidate Sekcja, Dział, Podklasa into one column PKD_2007
    # Assuming only one of them is populated or we prioritize them. 
    # Based on "There should always be only one value", it implies they are mutually exclusive or hierarchical.
    # Format 'Dział' as two digits with '.0' suffix if it exists

    # Format Dział -> it is string with 2 digits
    if 'Dział' in df_quarterly.columns:
        df_quarterly['Dział'] = df_quarterly['Dział'].astype(str).str.replace(r'\.0$', '', regex=True).replace('nan', None)
        df_quarterly['Dział'] = df_quarterly['Dział'].apply(lambda x: x.zfill(2) if x is not None else x)

    # Format Podklasa 2 digits + dot + 2 digits + dot + letter
    if 'Podklasa' in df_quarterly.columns:
        df_quarterly['Podklasa'] = df_quarterly['Podklasa'].astype(str).str.replace(r'^(\d{2})(\d{2})([A-Z])$', r'\1.\2.\3', regex=True).replace('nan', None)

    # Let's coalesce them.
    df_quarterly['PKD_2007'] = df_quarterly['Podklasa'].combine_first(df_quarterly['Dział']).combine_first(df_quarterly['Sekcja'])
    
    # Drop the original columns
    df_quarterly.drop(columns=['Sekcja', 'Dział', 'Podklasa'], inplace=True, errors='ignore')

    # Replace column Source_file with rok column (without) .xlsx and it will be int type
    df_quarterly['rok'] = df_quarterly['Source_File'].astype(str).str.replace('.xlsx', '', regex=False).astype(int)
    df_quarterly.drop(columns=['Source_File'], inplace=True)

    # Drop column Unnamed: 3
    df_quarterly.drop(columns=['Unnamed: 3'], inplace=True, errors='ignore')

    # To all columns except PKD_2007, rok add Przewidywana liczba pracujących as sufix for column names
    new_columns = {}
    for col in df_quarterly.columns:
        if col not in ['PKD_2007', 'rok']:
            new_columns[col] = f"Przewidywana liczba pracujących {col}"
    df_quarterly.rename(columns=new_columns, inplace=True)

    # Transpose data for these columns
    df_quarterly = df_quarterly.melt(id_vars=['PKD_2007', 'rok'], var_name='Wyszczegolnienie', value_name='Wartosc')

    display(df_quarterly.head())
    print(f"Successfully loaded {len(df_quarterly)} rows.")
except Exception as e:
    print(f"Error: {e}")

# Map the quarterly data (df_quarterly) into the combined_data structure
# First, normalize PKD_2007 codes to match with PKD 2025
df_quarterly["PKD_2007_normalized"] = df_quarterly["PKD_2007"].astype(str).str.strip().str.upper()
df_quarterly["pkd_formatted_no_letter"] = df_quarterly["PKD_2007_normalized"].str.replace(r"\.[A-Z]$", "", regex=True)
df_quarterly["pkd_letter"] = df_quarterly["PKD_2007"].str.extract(r"\.([A-Z])$")

# Map PKD 2007 to PKD 2025 using the mapping table
df_quarterly_mapped = df_quarterly.merge(
    mapowanie_pkd_2007_2025_with_og[["symbol_2007", "symbol_2025"]],
    how="left",
    left_on="pkd_formatted_no_letter",
    right_on="symbol_2007"
)

# Create pkd_2025 by combining symbol_2025 with the letter suffix
df_quarterly_mapped["symbol_2025"] = df_quarterly_mapped.apply(
    lambda row: row["symbol_2025"] + '.' + row["pkd_letter"] if pd.notna(row["pkd_letter"]) and pd.notna(row["symbol_2025"]) else row["symbol_2025"],
    axis=1
)

# Check for unmapped codes
unmapped_mask_quarterly = df_quarterly_mapped["symbol_2025"].isna()
if unmapped_mask_quarterly.any():
    missing_codes_quarterly = (
        df_quarterly_mapped.loc[unmapped_mask_quarterly, "PKD_2007_normalized"]
        .dropna()
        .unique()
        .tolist()
    )
    print(f"Warning: Niezmapowane kody PKD 2007 w danych kwartalnych: {missing_codes_quarterly}")
    df_quarterly_mapped = df_quarterly_mapped[~unmapped_mask_quarterly]

# Create the final quarterly dataframe in the same structure as combined_data
df_quarterly_final = df_quarterly_mapped[["rok", "symbol_2025", "Wyszczegolnienie", "Wartosc"]].copy()
df_quarterly_final = df_quarterly_final.rename(columns={
    "symbol_2025": "pkd_2025",
    "Wyszczegolnienie": "WSKAZNIK",
    "Wartosc": "wartosc"
})
df_quarterly_final

,PKD_2007,rok,Wyszczegolnienie,Wartosc
0,OG,2015,Przewidywana liczba pracujących Ogółem,3772931.0
1,A,2015,Przewidywana liczba pracujących Ogółem,68675.0
2,01,2015,Przewidywana liczba pracujących Ogółem,49170.0
3,01.11.Z,2015,Przewidywana liczba pracujących Ogółem,3742.0
4,01.12.Z,2015,Przewidywana liczba pracujących Ogółem,2.0


Successfully loaded 37805 rows.


,rok,pkd_2025,WSKAZNIK,wartosc
0,2015,OG,Przewidywana liczba pracujących Ogółem,3772931.0
1,2015,A,Przewidywana liczba pracujących Ogółem,68675.0
2,2015,01,Przewidywana liczba pracujących Ogółem,49170.0
3,2015,01.11.Z,Przewidywana liczba pracujących Ogółem,3742.0
4,2015,01.12.Z,Przewidywana liczba pracujących Ogółem,2.0
...,...,...,...,...
40700,2024,93.13.Z,Przewidywana liczba pracujących 250=>,-
40701,2024,96.99.Z,Przewidywana liczba pracujących 250=>,2
40702,2024,V,Przewidywana liczba pracujących 250=>,4
40703,2024,99,Przewidywana liczba pracujących 250=>,4


In [552]:
combined_data = pd.concat([wskaznik_finansowy_final, upadlosci_final,df_quarterly_final], ignore_index=True)
combined_data

,pkd_2025,WSKAZNIK,rok,wartosc
0,OG,EN Liczba jednostek gospodarczych,2005,"46 396,00"
1,OG,PEN Liczba rentownych jednostek gospodarczych,2005,"35 860,00"
2,OG,GS Przychody ogółem,2005,"1 679 774,30"
3,OG,PNPM Przychody netto,2005,<NA>
4,OG,GS (I) Przychody netto ze sprzedaży i zrównane...,2005,"1 614 314,24"
...,...,...,...,...
461798,93.13.Z,Przewidywana liczba pracujących 250=>,2024,-
461799,96.99.Z,Przewidywana liczba pracujących 250=>,2024,2
461800,V,Przewidywana liczba pracujących 250=>,2024,4
461801,99,Przewidywana liczba pracujących 250=>,2024,4


In [553]:
# Derive dictionary for WSKAZNIK values and attach their indices
unique_wskaznik = (
    combined_data["WSKAZNIK"].dropna().sort_values().unique()
)
wskaznik_dict = {value: idx for idx, value in enumerate(unique_wskaznik)}
wskaznik_dictionary_table = (
    pd.DataFrame(
        [(idx, value) for value, idx in wskaznik_dict.items()],
        columns=["WSKAZNIK_INDEX", "WSKAZNIK"]
    )
    .sort_values("WSKAZNIK_INDEX")
    .reset_index(drop=True)
)
combined_data["WSKAZNIK_INDEX"] = combined_data["WSKAZNIK"].map(wskaznik_dict)
combined_data.drop(columns=["WSKAZNIK"], inplace=True)
wskaznik_dictionary_table

,WSKAZNIK_INDEX,WSKAZNIK
0,0,C Środki pieniężne i pap. wart.
1,1,CF Nadwyżka finansowa
2,2,DEPR Amortyzacja
3,3,EN Liczba jednostek gospodarczych
4,4,GS (I) Przychody netto ze sprzedaży i zrównane...
5,5,GS Przychody ogółem
6,6,INV Zapasy
7,7,IO Wartość nakładów inwestycyjnych
8,8,IP Odsetki do zapłacenia
9,9,LTC Długoterminowe kredyty bankowe


In [554]:
combined_data

,pkd_2025,rok,wartosc,WSKAZNIK_INDEX
0,OG,2005,"46 396,00",3
1,OG,2005,"35 860,00",15
2,OG,2005,"1 679 774,30",5
3,OG,2005,<NA>,16
4,OG,2005,"1 614 314,24",4
...,...,...,...,...
461798,93.13.Z,2024,-,21
461799,96.99.Z,2024,2,21
461800,V,2024,4,21
461801,99,2024,4,21


In [555]:
pkd_2025_with_og['typ']

0         SEKCJA
1          DZIAŁ
2          GRUPA
3          KLASA
4       PODKLASA
          ...   
1771       DZIAŁ
1772       GRUPA
1773       KLASA
1774    PODKLASA
1775      OGÓŁEM
Name: typ, Length: 1776, dtype: object

In [556]:
# Build dictionary for PKD typ values and map indices
unique_typ_values = pkd_2025_with_og["typ"].dropna().unique()
typ_dict = {value: idx for idx, value in enumerate(unique_typ_values)}
pkd_typ_dictionary_table = (
    pd.DataFrame(
        [(idx, value) for value, idx in typ_dict.items()],
        columns=["TYP_INDEX", "typ"]
    )
    .sort_values("TYP_INDEX")
    .reset_index(drop=True)
)
pkd_2025_with_og["TYP_INDEX"] = pkd_2025_with_og["typ"].map(typ_dict)
pkd_2025_with_og.drop(columns=["typ"], inplace=True)
pkd_typ_dictionary_table

,TYP_INDEX,typ
0,0,SEKCJA
1,1,DZIAŁ
2,2,GRUPA
3,3,KLASA
4,4,PODKLASA
5,5,OGÓŁEM


In [557]:
pkd_2025_with_og

,symbol,nazwa,TYP_INDEX
0,A,"ROLNICTWO, LEŚNICTWO I RYBACTWO",0
1,01,"UPRAWY ROLNE, CHÓW I HODOWLA ZWIERZĄT, ŁOWIECT...",1
2,01.1,Uprawy rolne inne niż wieloletnie,2
3,01.11,"Uprawa zbóż innych niż ryż, roślin strączkowyc...",3
4,01.11.Z,"Uprawa zbóż innych niż ryż, roślin strączkowyc...",4
...,...,...,...
1771,99,ORGANIZACJE I ZESPOŁY EKSTERYTORIALNE,1
1772,99.0,Organizacje i zespoły eksterytorialne,2
1773,99.00,Organizacje i zespoły eksterytorialne,3
1774,99.00.Z,Organizacje i zespoły eksterytorialne,4


In [558]:
# Normalize PKD codes for consistent dictionary joins
def normalize_pkd_code(value):
    if pd.isna(value):
        return pd.NA
    code = str(value).strip().upper()
    if code.endswith('.0'):
        code = code[:-2]
    return code
pkd_2025_with_og["symbol_normalized"] = pkd_2025_with_og["symbol"].map(normalize_pkd_code)
combined_data["pkd_2025_normalized"] = combined_data["pkd_2025"].map(normalize_pkd_code)

In [559]:
# Add PKD identifiers and map them into the fact table
unique_pkd_values = pkd_2025_with_og["symbol_normalized"].dropna().sort_values().unique()
pkd_dict = {value: idx for idx, value in enumerate(unique_pkd_values)}
pkd_2025_with_og["PKD_INDEX"] = pkd_2025_with_og["symbol_normalized"].map(pkd_dict)
pkd_dictionary_table = (
    pkd_2025_with_og[["PKD_INDEX", "symbol", "nazwa", "TYP_INDEX"]]
    .drop_duplicates(subset=["PKD_INDEX"])
    .sort_values("PKD_INDEX")
    .reset_index(drop=True)
)
pkd_dictionary_table

,PKD_INDEX,symbol,nazwa,TYP_INDEX
0,0,01,"UPRAWY ROLNE, CHÓW I HODOWLA ZWIERZĄT, ŁOWIECT...",1
1,1,01.1,Uprawy rolne inne niż wieloletnie,2
2,2,01.11,"Uprawa zbóż innych niż ryż, roślin strączkowyc...",3
3,3,01.11.Z,"Uprawa zbóż innych niż ryż, roślin strączkowyc...",4
4,4,01.12,Uprawa ryżu,3
...,...,...,...,...
1759,1759,R,OPIEKA ZDROWOTNA I POMOC SPOŁECZNA,0
1760,1760,S,"DZIAŁALNOŚĆ ZWIĄZANA Z KULTURĄ, SPORTEM I REKR...",0
1761,1761,T,POZOSTAŁA DZIAŁALNOŚĆ USŁUGOWA,0
1762,1762,U,GOSPODARSTWA DOMOWE ZATRUDNIAJĄCE PRACOWNIKÓW ...,0


In [560]:
combined_data["PKD_INDEX"] = combined_data["pkd_2025_normalized"].map(pkd_dict)
combined_data

,pkd_2025,rok,wartosc,WSKAZNIK_INDEX,pkd_2025_normalized,PKD_INDEX
0,OG,2005,"46 396,00",3,OG,1756.0
1,OG,2005,"35 860,00",15,OG,1756.0
2,OG,2005,"1 679 774,30",5,OG,1756.0
3,OG,2005,<NA>,16,OG,1756.0
4,OG,2005,"1 614 314,24",4,OG,1756.0
...,...,...,...,...,...,...
461798,93.13.Z,2024,-,21,93.13.Z,1654.0
461799,96.99.Z,2024,2,21,96.99.Z,1727.0
461800,V,2024,4,21,V,1763.0
461801,99,2024,4,21,99,1738.0


In [561]:
missing_pkd_mask = combined_data["PKD_INDEX"].isna()
if missing_pkd_mask.any():
    missing_pkd_codes = (
        combined_data.loc[missing_pkd_mask, "pkd_2025_normalized"]
        .dropna()
        .unique()
        .tolist()
    )
    # raise ValueError(f"Niezmapowane kody PKD: {missing_pkd_codes}")
    print(f"Warning: Niezmapowane kody PKD: {missing_pkd_codes}")
    combined_data = combined_data[~missing_pkd_mask]

In [562]:
columns_to_drop = [col for col in ["pkd_2025", "pkd_2025_normalized"] if col in combined_data.columns]
if columns_to_drop:
    combined_data.drop(columns=columns_to_drop, inplace=True)
if "symbol_normalized" in pkd_2025_with_og.columns:
    pkd_2025_with_og.drop(columns=["symbol_normalized"], inplace=True)

C:\Users\tusiu\AppData\Local\Temp\ipykernel_2196\95792252.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_data.drop(columns=columns_to_drop, inplace=True)


In [563]:
mapowanie_pkd_2007_2025_with_og

,symbol_2007,symbol_2025
0,01,01
1,01.1,01.1
2,01.11,01.11
3,01.12,01.12
4,01.13,01.13
...,...,...
1021,R,S
1022,S,T
1023,T,U
1024,U,V


In [564]:
from decimal import Decimal, InvalidOperation
import pandas as pd

def str_to_decimal(val):
    if isinstance(val, str):
        val_clean = val.replace('\xa0', '').replace(' ', '').replace(',', '.')
        # Handle common non-numeric values like empty strings or dashes
        if val_clean in ['', '-', '–']:
            return None
        try:
            return Decimal(val_clean)
        except InvalidOperation:
            print(f"Warning: Invalid value encountered: '{val}'")
            return None
    return val
combined_data['wartosc'] = combined_data['wartosc'].apply(str_to_decimal)

C:\Users\tusiu\AppData\Local\Temp\ipykernel_2196\2629187910.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_data['wartosc'] = combined_data['wartosc'].apply(str_to_decimal)


In [565]:
print(combined_data['wartosc'].apply(type).value_counts())


wartosc
<class 'decimal.Decimal'>                320476
<class 'pandas._libs.missing.NAType'>     98084
<class 'int'>                             28233
<class 'float'>                            7102
<class 'NoneType'>                         2278
Name: count, dtype: int64


In [566]:
import os

# Save final tables to CSV files
# Create results directory if it doesn't exist
os.makedirs(r"..\results", exist_ok=True)

wskaznik_dictionary_table.to_csv(r"..\results\wskaznik_dictionary.csv", sep=";", encoding="utf-8", index=False)
pkd_dictionary_table.to_csv(r"..\results\pkd_dictionary.csv", sep=";", encoding="utf-8", index=False)
pkd_typ_dictionary_table.to_csv(r"..\results\pkd_typ_dictionary.csv", sep=";", encoding="utf-8", index=False)
combined_data.to_csv(r"..\results\kpi-value-table.csv", sep=";", encoding="utf-8", index=False)

print("All tables saved successfully!")

All tables saved successfully!


# final tables
* wskaznik_dictionary_table - dim
* combined_data - fact
* pkd_dictionary_table - dim pkd
* pkd_typ_dictionary_table - dim hierarchy